In [2]:
#!/Users/mahmoudshepero/anaconda3/bin/python3

import python as SP
from osgeo import ogr, osr
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import extractFiles as ex
import cars as cc1
from matplotlib.colors import Normalize
import matplotlib as mlp
mlp.use("Agg")
import random as rnd
import matplotlib.animation as manimation

In [3]:
# load the buildings layer "LänmaterietLayer"
InputFileLocation = r'../Data/NewFolder/buildings-epsg3006.shp' # location
shapefile1 = InputFileLocation
buildingDS1 = ogr.Open(shapefile1)
buildingDS2 = ogr.Open(shapefile1)
buildingDS3 = ogr.Open(shapefile1)


# print the number of layers
print("Layer names:", SP.list_of_layers(buildingDS1))


# load the first layer
buildingsLayer = buildingDS1.GetLayer()

# check that the spatial reference is a metric spatial reference
print("spatial reference system: ",buildingsLayer.GetSpatialRef().ExportToWkt())

# filter to remove buildings with area less than 10m2
buildingsLayer.SetAttributeFilter("OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10")
buildingsLayer.ResetReading() # reset counting of the filter

# print the layer fields
print("Layer Fields:", SP.get_layer_fields(buildingsLayer))

# print the tags from the interesting fieldDefn
print("Field tags: ", SP.get_field_tags(buildingsLayer, "ANDAMAL_1"))


## RESIDENTIAL LAYER
# load the layer to residential
ResbuildingsLayer = buildingDS1.GetLayer()
# we are interested in the layer residential codes 133 and 135, 199
ResbuildingsLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (133, 135, 199)"
)
ResbuildingsLayer.ResetReading() # reset counting of the filter
print("Number of residential features: ", ResbuildingsLayer.GetFeatureCount())


## WORKPLACE LAYER
# load the layer to workplace
workplacesLayer = buildingDS2.GetLayer()
# workplaces are 240:299, 302, 304, 307, 311, 319, 322, 499
workplacesLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (240, 242, 243, 247, 248, 249, 252, 253, 299, "
"302, 304, 307, 311, 319, 322, 499)"
)
workplacesLayer.ResetReading() # reset counting of the filter
print("Number of workplace features: ", workplacesLayer.GetFeatureCount())


## OTHERS LAYER
# load the layer to other
otherLayer = buildingDS3.GetLayer()
# Otherplaces (leisure and shopping) are 301, 309, 313, 316, 317, 320, 399, 799
otherLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (301, 309, 313, 316, 317, 320, 399, 799)"
)
otherLayer.ResetReading() # reset counting of the filter
print("Number of other features: ", otherLayer.GetFeatureCount())


# Save the layers
SP.create_buffer_and_projectLayer(ResbuildingsLayer, 3006, 3006, "residentialLayer", bufferDist = 0)
SP.create_buffer_and_projectLayer(workplacesLayer, 3006, 3006, "workLayer", bufferDist = 0)
SP.create_buffer_and_projectLayer(otherLayer, 3006, 3006, "otherLayer", bufferDist = 0)




Layer names: ['buildings-epsg3006']
spatial reference system:  PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]
Layer Fields: ['DETALJTYP', 'ANDAMAL_1', 'ANDAMAL_1T', 'area']
Field tags:  {130, 131, 132, 133, 135, 399, 799, 299, 301, 302, 304, 305, 306, 307, 308, 309, 310, 313, 314, 699, 316, 317, 318, 319, 320, 321, 322, 315, 199, 599, 240, 242, 499, 243, 246, 247, 249, 252, 253}
Number of residential features:  5406
Number of workplace features:  2697
Number of other features:  1391


In [4]:
# Open Parking layer
InputFileLocation = r'../Data/NewFolder/OSM-epsg3600.shp' # location
shapefile2 = InputFileLocation
parkingDB = ogr.Open(shapefile2)
parkingLayer = parkingDB.GetLayer()

# print the parking layer tags
print("Amenity Field tags: ", SP.get_field_tags(parkingLayer, "amenity"))
print("Building Field tags: ", SP.get_field_tags(parkingLayer, "building"))
# Remove None features
parkingLayer.SetAttributeFilter( "NOT OGR_GEOM_WKT LIKE 'None%' AND OGR_GEOM_AREA > 10"
  "AND (amenity IN ('parking', 'parking_space') OR building in ('garage', 'garages'))")
parkingLayer.ResetReading() # reset counting of the filter
print("Number of parking features: ", parkingLayer.GetFeatureCount())


# Open Buffered layer
InputFileLocation = r'UppsalaParkingBuffer100meter3006.shp' # location
shapefile3 = InputFileLocation
parkingBuffer = ogr.Open(shapefile3)
parkingBufferLayer = parkingBuffer.GetLayer()


Amenity Field tags:  {'concert_hall', 'parking', 'social_facility', 'cafe', 'shelter', 'car_rental', 'waste_disposal', 'fountain', 'doctors', 'fast_food', 'recycling', 'hospital', 'clinic', 'university', 'parking_space', 'events_venue', 'fire_station', 'boat_storage', 'animal_shelter', 'restaurant', 'grave_yard', 'community_centre', 'nursing_home', 'toilets', 'car_wash', 'vehicle_inspection', 'monastery', 'fuel', 'motorcycle_parking', 'prison', 'bar', 'childcare', 'social_centre', 'school', 'nightclub', 'library', None, 'animal_training', 'place_of_worship', 'bicycle_parking', 'theatre', 'kindergarten'}
Building Field tags:  {'barn', 'warehouse', 'hangar', 'industrial', 'sport', 'cathedral', 'stable', 'garages', 'hospital', 'residential', 'detached', 'church', 'university', 'yes', 'garage', 'roof', 'boathouse', 'house', 'preschool', 'commercial', 'hotel', 'office', 'apartments', 'terrace', 'shed', 'farm', 'school', 'public', 'farm_auxiliary', 'hut', 'silo', 'service', 'retail', 'greenh

In [ ]:
# Save the figure
fig = plt.figure(figsize = (500,500))
SP.plot_features(parkingLayer, "parkingPlot.pdf", '#555577')
SP.plot_features(otherLayer, "parkingPlot.pdf", '#118877')
SP.plot_features(workplacesLayer, "parkingPlot.pdf", '#992266')
SP.plot_features(ResbuildingsLayer, "parkingPlot.pdf", '#228811')
fig.savefig("parkingPlot1.pdf")


In [5]:
# Calculate intersection of the buffered parking lot layer with the buidlings layers
areaPercentage = SP.get_percentage_of_area_types(parkingBufferLayer,
                [ResbuildingsLayer, workplacesLayer, otherLayer])
np.savetxt("areaPercentage.txt", areaPercentage)

# Get the areas of the parking lots
parkingLotAreas = SP.get_features_areas(parkingLayer)

In [6]:
# Get station IDs from the data set
ID = SP.get_field_tags(parkingLayer, "osm_way_id")
# Assert there are no duplicate IDs
assert len(ID) == parkingLayer.GetFeatureCount(), "there are duplicate IDs, create your own IDs for the stations" 


In [13]:
stations = SP.create_charging_stations(ID, 
                               parkingLotAreas, 
                               areaPercentage,
                               areaPerCar = 15,     
                               charging_status = True, 
                               charging_power = 3.7)

In [241]:
# Number of state 0 stations
print("number of home stations:", 
      len(list(iter.filterfalse(lambda x: not(x.state ==0),stations))))
# Number of state 1 stations
print("number of work stations:", 
      len(list(iter.filterfalse(lambda x: not(x.state ==1),stations))))
# Number of state 2 stations
print("number of other stations:", 
      len(list(iter.filterfalse(lambda x: not(x.state ==2),stations))))


number of home stations: 1350
number of work stations: 979
number of other stations: 733


In [242]:
# Number of parking palaces for state 0 stations
print("number of home stations:", 
      sum(list(map(lambda x: x.maximumOccupancy, list(iter.filterfalse(lambda x: not(x.state ==0),stations))))))
# Number of parking palaces for state 1 stations
print("number of work stations:", 
      sum(list(map(lambda x: x.maximumOccupancy, list(iter.filterfalse(lambda x: not(x.state ==1),stations))))))
# Number of parking palaces for state 2 stations
print("number of other stations:", 
      sum(list(map(lambda x: x.maximumOccupancy, list(iter.filterfalse(lambda x: not(x.state ==2),stations))))))



number of home stations: 36639
number of work stations: 50552
number of other stations: 15982


In [360]:
# Create EVs
numberCars = 10000

EVs = [cc1.EV(currentLocation = None, currentState = None, batteryCharge = 0.0,
                 batteryCapacity = 0.0, trips = 0) for i in range(numberCars)]
# Randomly allocate the initial location of EVs
list(map(lambda x: x.inital_conditions(stations,0), EVs))

# print location
list(map(lambda x: x.currentLocation, EVs))

['442886299-1',
 '437723098-2',
 '127460065-0',
 '33632836-1',
 '247519955-0',
 '220110163-1',
 '265029567-0',
 '104145830-1',
 '15217727-1',
 '488528259-0',
 '96680127-0',
 '99728530-1',
 '96689282-2',
 '65730397-1',
 '98710878-2',
 '58227810-2',
 '95806876-0',
 '439890445-1',
 '102579954-1',
 '137320451-1',
 '110739348-2',
 '83075450-1',
 '164081093-0',
 '95806869-2',
 '95806870-0',
 '247753960-1',
 '82122194-0',
 '99728471-0',
 '127212150-2',
 '341201668-0',
 '134643427-1',
 '99192286-2',
 '112003174-0',
 '96131306-2',
 '103271529-0',
 '438534317-2',
 '436027733-0',
 '442886295-2',
 '104145829-1',
 '100015378-2',
 '102693347-2',
 '33828353-0',
 '488450160-1',
 '436172508-0',
 '99511195-0',
 '37414354-1',
 '439036531-2',
 '544508707-1',
 '105289130-2',
 '454859794-2',
 '110364108-2',
 '103663336-2',
 '110739450-1',
 '110364085-0',
 '265030599-0',
 '105599713-0',
 '103271529-0',
 '104145845-2',
 '134643433-1',
 '265029565-1',
 '253365835-1',
 '226093953-2',
 '99995424-1',
 '88523593-0

In [7]:
# load transition Matrix
weekdayChain = ex.readMatrixfiles("../TransitionMatrix/*weekday*.txt")
# define the tranistion Matrix
weekday = cc1.Markov(weekdayChain)

In [361]:
# Setup the simulation Model
simulationCase = cc1.Simulation(stations,
                            EVs,
                            weekday)

lengthOfSimulation = 2880 # timesteps, i.e. 2 days.

# Estimate the electric load
load = simulationCase.simulate_model(lengthOfSimulation)


In [362]:
np.savetxt("load1.txt",np.asarray(load))

In [18]:
load = np.loadtxt("load1.txt", ndmin = 2)
np.asarray(load).shape

(2880, 3062)

In [19]:
def collect_stations_results(ID, results, stations):
    results_temp = np.copy(results)
    lengthOfSimulation = results_temp.shape[0]
    IDs = list(ID)
    stationsIDs = list(map(lambda x: x.ID, stations))
    final_results = np.zeros((lengthOfSimulation,len(IDs)))
    
    for i in enumerate(IDs):
        columns = [idx for idx in range(len(stationsIDs)) if IDs[i[0]] in stationsIDs[idx]]
        temp =  results_temp[:,columns].sum(1).reshape(lengthOfSimulation,1)
        final_results[:,list([i[0]])] = temp
    return(final_results)

final_results = collect_stations_results(ID, load, stations)        
final_results.shape

(2880, 1718)

In [20]:
np.savetxt("finalResults.txt",final_results[1440:,:])

In [26]:
plotted = final_results[1440:,:]
plotted[0,0]

0.0

In [22]:

def norm_cmap(values, cmap, vmin=None, vmax=None):
    """
    Normalize and set colormap
    
    Parameters
    ----------
    values : Series or array to be normalized
    cmap : matplotlib Colormap
    normalize : matplotlib.colors.Normalize
    cm : matplotlib.cm
    vmin : Minimum value of colormap. If None, uses min(values).
    vmax : Maximum value of colormap. If None, uses max(values).
    
    Returns
    -------
    n_cmap : mapping of normalized values to colormap (cmap)
    https://ocefpaf.github.io/python4oceanographers/blog/2015/08/24/choropleth/
    
    """
    mn = np.amin(values)
    
    mx = np.amax(values)
    norm = Normalize(vmin=mn, vmax=mx)
    n_cmap = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    return(n_cmap)



cmap = norm_cmap(plotted, cmap='YlOrRd')

def f(x):
    rgba = cmap.to_rgba(x)
    return(mlp.colors.to_hex(rgba))




In [25]:

def plot(layer, color,filename):
    fig = plt.figure(figsize = (500,500))
    i = 0
    for feature in layer:
        ring = feature.GetGeometryRef()
        coord = ring.GetGeometryRef(0)
        #coord1 = coord.GetGeometryRef(0)
        points = coord.GetPoints()
        x, y = zip(*points)
        plt.fill(x, y, f(color[i]))
        i += 1

    plt.xlabel("Easting (m)", fontsize=25)
    plt.ylabel("Northing (m)", fontsize=25)
    plt.axis('equal')
    layer.ResetReading()
    fig.savefig(filename)
    
plot(parkingLayer, list(plotted[1000,:]), "results1000.pdf")

In [ ]:
FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib',
                comment='Movie support!')
writer = FFMpegWriter(fps=30, metadata=metadata)

fig = plt.figure()


with writer.saving(fig, "ResultsDay.mp4", 100):
    for i in range(1440):
        plt.clf()
        j = 0
        for feature in parkingLayer:
            ring = feature.GetGeometryRef()
            coord = ring.GetGeometryRef(0)
            points = coord.GetPoints()
            x, y = zip(*points)
            plt.fill(x, y, f(plotted[i,j]))
            plt.xlim(650000, 652000)
            plt.ylim(6635000, 6640000)
            plt.xlabel("Easting (m)", fontsize=25)
            plt.ylabel("Northing (m)", fontsize=25)
            plt.axis('equal')
            plt.title(str(int(i/60)).zfill(2)+":"+str(i%60).zfill(2), fontsize=30)
            j += 1
        writer.grab_frame()
        parkingLayer.ResetReading()
        

In [21]:
i=600
str(int(i/60)).zfill(2)+":"+str(i%60).zfill(2)

'10:00'

In [267]:
import imp; imp.reload(SP); 
imp.reload(cc1)

<module 'cars' from '/Users/mahmoudshepero/Documents/PhD/Models/EVSpatialModelPython/PythonCode/cars.py'>